In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
from sacred import Experiment
from sacred.commands import print_config
from sacred.observers import FileStorageObserver

ex = Experiment('PHME20', interactive=True)
# ex.observers.append(FileStorageObserver('./lgbm_logs/'))

In [ ]:
@ex.config
def configuration_settings():
    fill_missing_ = True
    scale_type = "standard"
    window_size_ = 5
    train_model = True
    eval_model = True
    subsample = 10 # for sampling
    stride = 5 # for stride
    
    num_leaves_ = -1
    learning_rate_ = 0.1
    n_estimators_ = 100
    

In [ ]:
### import data
data_df_1 = pd.read_csv("../../data/training_validation_1.csv")
data_df_2 = pd.read_csv("../../data/training_validation_2.csv")
data_df_3 = pd.read_csv("../../data/model_refinement.csv")

train_df = pd.concat([data_df_1, data_df_2, data_df_3], axis=0) # Merge data frames
# train_df = data_df_2.filter(regex="vCnt|value")

train_df['runId'] = 1000 * train_df['class'] + train_df['run']

labels = train_df['class']
runs = train_df['runId']

run_df = train_df[['class', 'runId']].copy()
run_df.drop_duplicates(inplace=True)
run_df.reset_index(inplace=True)
del run_df['index']

# del train_df['class']
del train_df['run']

train_df.shape

In [ ]:
% skip 
sensor_list = list(train_df.columns)
sensor_list.remove('runId')
sensor_list.remove('class')
len(sensor_list)

In [ ]:
sensor_list_xgb = ['Humidity_value',
 'TotalMemoryConsumption_value',
 'SmartMotorSpeed_value',
 'TotalCpuLoadNormalized_value',
 'Temperature_value',
 'ProcessMemoryConsumption_value',
 'DurationRobotFromFeederToTestBench_value',
 'CpuTemperature_value',
 'FuseCycleDuration_value',
 'FuseTestResult_vCnt',
 'IntensityTotalThermoImage_value',
 'Pressure_value',
 'ValidFrame_vCnt',
 'DurationPickToPick_value',
 'DurationTestBenchClosed_value',
 'VacuumValveClosed_value',
 'ProcessCpuLoadNormalized_value',
 'FuseOutsideOperationalSpace_vCnt',
 'Pressure_vCnt',
 'FeederAction2_vCnt',
 'FuseTestResult_value',
 'NumberFuseEstimated_value',
 'Vacuum_vCnt',
 'LightBarrierActiveTaskDuration1_value',
 'FusePicked_vCnt',
 'DurationTestBenchClosed_vCnt',
 'FeederAction4_vCnt',
 'DurationPickToPick_vCnt',
 'EPOSCurrent_value',
 'VacuumFusePicked_value',
 'FuseHeatSlopeOK_value',
 'EPOSCurrent_vCnt']

len(sensor_list_xgb)

In [ ]:
sensor_list_lgbm = ['TotalMemoryConsumption_value',
 'Humidity_value',
 'Temperature_value',
 'ProcessMemoryConsumption_value',
 'TotalCpuLoadNormalized_value',
 'SmartMotorSpeed_value',
 'FuseCycleDuration_value',
 'NumberFuseDetected_value',
 'DurationRobotFromFeederToTestBench_value',
 'Pressure_value',
 'LightBarrierPassiveTaskDuration1_vCnt',
 'FeederBackgroundIlluminationIntensity_value',
 'Vacuum_value',
 'Pressure_vCnt',
 'VacuumFusePicked_vCnt',
 'ProcessCpuLoadNormalized_value',
 'DurationPickToPick_vCnt',
 'EPOSPosition_value',
 'NumberFuseEstimated_value',
 'TemperatureThermoCam_value',
 'VacuumValveClosed_vCnt',
 'FusePicked_vCnt',
 'FusePicked_value',
 'FuseHeatSlopeNOK_value',
 'FuseCycleDuration_vCnt',
 'FuseHeatSlopeOK_value',
 'IntensityTotalThermoImage_value',
 'FuseIntoFeeder_vCnt',
 'ValidFrameOptrisPIIRCamera_vCnt',
 'FeederAction4_vCnt']

len(sensor_list_lgbm)

In [ ]:
sensor_list = sensor_list_lgbm

In [ ]:
from catboost import CatBoostClassifier
from ngboost import NGBClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_df.head()

In [ ]:
# split a sequence into samples
def create_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)


In [ ]:
def create_dataset_for_run(df, ws):
#     data_data = np.empty((0, ws * len(sensor_list))) # for 1D
#     data_data = np.empty((0, ws, len(sensor_list))) # for 2D
#     data_data = np.empty((0, len(sensor_list), ws)) # for 2D
#     label_data = np.empty((0, 1))

    sensors_df = df.filter(sensor_list)

    # Calculate seq of windows_size len
    seq = create_sequence(sensors_df.values, n_steps=ws)
#     seq = np.transpose(seq, axes=(0, 2, 1))
    seq_count = seq.shape[0]
    seq = seq.reshape((seq_count, -1)) # for 1D

    # add new seq to data_data array
#     data_data = np.vstack((data_data, seq))

    # Calculate RULS
    labels = df['class'].values[:seq_count]

    # add rul to rul_data array
#     rul_data = np.vstack((rul_data, ruls))

# TODO: What is RUL_Max in this context?

#     print ("Shape:", seq.shape, labels.shape)
    return seq, labels


 

In [ ]:
# Globals

# Main sensor list. 
# List shall include other features such as operating conditions. 
_subsample = -1
_stride = -1

# scale_type = "standard"
# window_size = 50
# cv_fold = 3

# train_model = True
# eval_model = True
verbose = False

In [ ]:
# TODO: X_t, X_tp1, y_t, y_tp1 should be calculated per run.  
# TODO: Then should be merged into one X_t, X_tp1, y_t, y_tp1.
def create_datasets(df, ws):
    
    run_list = df['runId'].unique()

    X_df_list = []
    y_df_list = []
    
    for r in run_list:
        r_df = df[df['runId'] == r]
#         print ("--> r: ", r, r_df.shape)
        sensor_data, label_data = create_dataset_for_run(r_df, ws)

        # Post Processing for the model

        # Padding for model input 
        padded_sensor_data = sensor_data.copy() #np.hstack((sensor_data, np.zeros((sensor_data.shape[0], 2)))) # for AE     

        # Calculate X(t) and X(t+1) for model input/output 
        X_t = padded_sensor_data[:]

        # Calculate y(t) and y(t+1) for model input/output 
        y_t = label_data[:]

        X_df_list.append(pd.DataFrame(X_t))
        y_df_list.append(pd.DataFrame(y_t))
    
    X_t = pd.concat(X_df_list, axis=0) # Merge data frames
    y_t = pd.concat(y_df_list, axis=0) # Merge data frames

    return X_t.values, y_t.values.flatten()


In [ ]:
# cv = StratifiedKFold(n_splits=4, shuffle=True)

In [ ]:
fillna_list = [True, False]
ws_list = [1, 2, 3, 4, 5, 10, 15, 20, 25, 30]

params_num_leaves = [5, 10, 15, 20, 30, 40, 50]
param_learning_rate = [0.1, 0.01, 0.001]
params_n_estimators = [100, 200, 300, 400, 500, 1000]

total_runs = len(fillna_list) * len(ws_list) * len(params_num_leaves) * len(param_learning_rate)  * len(params_n_estimators)

total_runs

In [ ]:
run_counter = 0
results_df = pd.DataFrame()

@ex.main
def ex_main(_run, fill_missing_, scale_type, window_size_, train_model, eval_model, subsample, stride, num_leaves_, learning_rate_, n_estimators_):

    global total_runs
    global run_counter
    global results_df
    
    print ('===========================================================================')
    print ('Run:', run_counter+1, '/', total_runs)
    print ("Fill missing:", fill_missing_, "Window Size:", window_size_, "LGBM Params:", num_leaves_, learning_rate_, n_estimators_)

    run_counter += 1
    
    acc_sum = 0
    f1_sum = 0

    cv = StratifiedKFold(n_splits=3, shuffle=True)

    for fold, (training_indices, validation_indices) in enumerate(cv.split(run_df['runId'], run_df['class'])):
        print ("--> Fold: ", fold)

        training_runIds = run_df.loc[training_indices]['runId']
        validation_runIds = run_df.loc[validation_indices]['runId']

        X_train_df = train_df[train_df['runId'].isin(training_runIds)].copy()
        X_val_df = train_df[train_df['runId'].isin(validation_runIds)].copy()

        if (fill_missing_):
            X_train_df.fillna(method='backfill', inplace=True)
            X_val_df.fillna(method='backfill', inplace=True)

            X_train_df.fillna(-1, inplace=True)
            X_val_df.fillna(-1, inplace=True)

        X_train, y_train = create_datasets(X_train_df, window_size_)
        X_val, y_val = create_datasets(X_val_df, window_size_)

    #     print ("Data shape", X_train_df.shape, X_val_df.shape)
        print ("Train data shape:", X_train.shape, y_train.shape)
        print ("Val data shape:", X_val.shape, y_val.shape)

        model = LGBMClassifier(num_leaves=num_leaves_, learning_rate=learning_rate_, n_estimators=n_estimators_)
        model.fit(X_train, y_train)

#         m = np.argmax(model.feature_importances_)
#         print (sensor_list[m], model.feature_importances_[m])

        pred = model.predict(X_val)

        acc_val = accuracy_score(pred, y_val)
        f1_val = f1_score(pred, y_val, average='weighted')

        acc_sum += acc_val
        f1_sum += f1_val

        print ("Fold:", fold, "ACC:", acc_val, "F1:", f1_val)

    print ()
    print ("Avg ACC:", acc_sum / 3.0, "Avg F1:", f1_sum / 3.0)

    result = {
        'ML_Algorithm': 'LGBM',
        'Fill missing': fill_missing_, 
        'Window Size': window_size_, 
        'Params_num_leaves': num_leaves_, 
        'Params_learning_rate': learning_rate_, 
        'Params_n_estimators': n_estimators_,        
        'ACC': acc_sum / 3.0,
        'F1': f1_sum / 3.0,
    }

    results_df = results_df.append(result, ignore_index=True)
    results_df.to_excel("lgbm_results.xlsx")    

In [ ]:
for fnal in fillna_list:
    for wsl in ws_list:
        for pnl in params_num_leaves:
            for plr in param_learning_rate:
                for pne in params_n_estimators:
                    ex.run(config_updates={
                        'fill_missing_': fnal,
                        'window_size_': wsl,
                        'num_leaves_': pnl,
                        'learning_rate_': plr,
                        'n_estimators_': pne,
                    })

